In [6]:
# Note These probably need to be run at least the first time you open the network (not sure if after that)
# !pip install fastai
# !apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
# !pip install Pillow==4.0.0
# !pip install image
# !pip install torchtext==0.2.3

    100% |████████████████████████████████| 51kB 2.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
import torch

In [0]:
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
from fastai.vision import *

ImportError: ignored